In [367]:
import pandas as pd
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import locale

In [368]:
dfHouses = pd.read_csv(r'casas-pro.csv')
dfDepartments = pd.read_csv(r'departamentos-pro.csv')

In [369]:
#Remove MN, $, USD and convert dollar to MX 
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
dfHouses['precio'] = dfHouses['precio'].apply(lambda x: (locale.atof(x[3:]) if x[0] == 'M' else (locale.atof(x[1:]) if x[0] == '$' else (locale.atof(x[4:]) * 20.58 if x[0] == 'U'else locale.atof(x)))))

dfDepartments['precio'] = dfDepartments['precio'].apply(lambda x: (locale.atof(x[3:]) if x[0] == 'M' else (locale.atof(x[1:]) if x[0] == '$' else (locale.atof(x[4:]) * 20.58 if x[0] == 'U'else locale.atof(x)))))

In [370]:
#Remove M²
dfHouses['mets'] = dfHouses['mets'].apply(lambda x: locale.atof(x[:len(x) - 3]) if x[len(x) - 1 ] == '²' else locale.atof(x))
dfDepartments['mets'] = dfDepartments['mets'].apply(lambda x: locale.atof(x[:len(x) - 3]) if x[len(x) - 1 ] == '²' else locale.atof(x))

In [371]:
#Get all "refugio" elements
dfHouses = dfHouses[dfHouses['titulo'].str.contains("refugio", na = False, case = False)]
dfHouses.reset_index(drop=True, inplace=True)

dfDepartments = dfDepartments[dfDepartments['titulo'].str.contains("refugio", na = False, case = False)]
dfDepartments.reset_index(drop=True, inplace=True)

In [372]:
#dfHouses.sort_values(by = ['mets'], ascending=False)

In [373]:
dfHouses['mets'] = dfHouses['mets'].replace([2175000.0],2175)

In [374]:
#get meters and prices 

houseMeters = dfHouses.loc[:,['mets']]
housePrices = dfHouses.loc[:,['precio']]

departmentMeters = dfDepartments.loc[:,['mets']]
departmentPrices = dfDepartments.loc[:,['precio']]


In [375]:
#get training and test frame

houseMetersTraining, houseMetersTest, housePricesTraining, housePricesTest = train_test_split(houseMeters, housePrices, test_size = 0.1, random_state = 0)
departmentMetersTraining, departmentMetersTest, departmentPricesTraining, departmentPricesTest = train_test_split(departmentMeters, departmentPrices, test_size = 0.1, random_state = 0)


In [376]:
houseModel = linear_model.LinearRegression()
departmentModel = linear_model.LinearRegression()

In [377]:
houseModel.fit(houseMetersTraining,housePricesTraining)
departmentModel.fit(departmentMetersTraining, departmentPricesTraining)

LinearRegression()

In [378]:
#Reset index
houseMetersTest.reset_index(drop=True, inplace=True)
housePricesTest.reset_index(drop=True, inplace=True)

departmentMetersTest.reset_index(drop=True, inplace=True)
departmentPricesTest.reset_index(drop=True, inplace=True)


In [379]:
print(len(houseMetersTest))
print('Casas')
print('Metros  |  Precio Real  |  Precio predicho')
print('--------+---------------+-----------------')
for i in range(10):
    print('{:>6}'.format(houseMetersTest.loc[i][0]) + '  |' + '{:>12}'.format(housePricesTest.loc[i][0]) + '   |' + '{:>15.3f}'.format(houseModel.predict(pd.DataFrame([houseMetersTest.loc[i][0]]))[0][0]))

213
Casas
Metros  |  Precio Real  |  Precio predicho
--------+---------------+-----------------
 102.0  |   2340000.0   |    3005323.585
 136.0  |   2750000.0   |    3049293.475
 307.0  |   5000000.0   |    3270436.162
 134.0  |   2950000.0   |    3046707.011
 135.0  |   2950000.0   |    3048000.243
 110.0  |   2465000.0   |    3015669.441
 131.0  |   2890000.0   |    3042827.315
  99.0  |   1850000.0   |    3001443.888
 200.0  |   2302300.0   |    3132060.329
 120.0  |   2130000.0   |    3028601.762


In [380]:
print(len(departmentMetersTest))
print('Departamentos')
print('Metros  |  Precio Real  |  Precio predicho')
print('--------+---------------+-----------------')

for i in range(len(departmentMetersTest)):
    print('{:>6}'.format(departmentMetersTest.loc[i][0]) + '  | ' + '{:>10}'.format(departmentPricesTest.loc[i][0]) + '    |' + '{:>13.3f}'.format(departmentModel.predict(pd.DataFrame([departmentMetersTest.loc[i][0]]))[0][0]))

6
Departamentos
Metros  |  Precio Real  |  Precio predicho
--------+---------------+-----------------
 157.0  |    15900.0    |    16302.880
 190.0  |    18000.0    |    17589.109
  64.0  |    14000.0    |    12678.052
 100.0  |    12900.0    |    14081.211
 250.0  |    23000.0    |    19927.707
 160.0  |    18000.0    |    16419.810
